# Compute adonis2

In [2]:
library(data.table)
library(tidyverse)
library(vegan)

In [ ]:
out_dir <- "out_adonis2/"

metadata <- read_tsv(
    "../data_March23/metadata_formatted_Mar23.tsv", 
    show_col_types = FALSE)
mpa_profiles <- read_tsv(
    "../data_March23/metaph4_formatted_asinsqrt.tsv", 
    show_col_types = FALSE)

combs <- list(
    "stage_class" = c(
        "control_vs_adenoma",
        "adenoma_vs_0__I__II__III__IV",
        "control_vs_0__I__II__III__IV",
        "0__I__II_vs_III__IV",
        "0__I__II__III_vs_IV"
    ),
    "tumor_location_class" = c(
        "right_sided_vs_left_sided",
        "right_sided__transverse_vs_left_sided",
        "right_sided_vs_transverse__left_sided"
    )
    
)

out_df <- data.frame()
j <- 1

for (curr_name in names(combs)) {
    
    for (comb in combs[[curr_name]]) {
        
        comb1 <- unlist(
        strsplit(
                unlist(
                    strsplit(
                        comb, split="_vs_"
                    )
                )[1], split="__")
            )
        comb2 <- unlist(
            strsplit(
                unlist(
                    strsplit(
                        comb, split="_vs_"
                    )
                )[2], split="__")
        )

        cat("\n")
        print(comb1)
        print(comb2)
        
        tmp_metadata <- metadata[which(
            metadata[, curr_name, drop=TRUE] %in% c(comb1, comb2)), ]
        print(dim(metadata))
        print(dim(tmp_metadata))
        
        tmp_metaph <- as.data.frame(mpa_profiles)
        rownames(tmp_metaph) <- tmp_metaph$feat_name; tmp_metaph$feat_name <- NULL
        tmp_metaph <- as.data.frame(t(tmp_metaph))
        tmp_metaph <- tmp_metaph[tmp_metadata$sample_id, ]
        tmp_metaph <- tmp_metaph[which(rowSums(tmp_metaph > 0) > 20), ]
        tmp_metadata <- tmp_metadata[which(tmp_metadata$sample_id %in% rownames(tmp_metaph)), ]
        
        tmp_metadata[which(tmp_metadata[, curr_name, drop=TRUE] %in% comb1), "target_class"] <- "comb1"
        tmp_metadata[which(tmp_metadata[, curr_name, drop=TRUE] %in% comb2), "target_class"] <- "comb2"
        
        perm <- how(nperm = 999)
        setBlocks(perm) <- with(tmp_metadata, study_name)

        permanova <- adonis2(tmp_metaph ~ target_class,
            data = tmp_metadata,
            permutations=perm,
            method = "bray")
        
#         pval <- permanova[["aov.tab"]]["target_class", "Pr(>F)"]
    
        tmp_out <- as.data.frame.matrix(permanova)
        tmp_out$rownames <- rownames(tmp_out)
        
        tmp_out$comb <- curr_name
        
        tmp_out$comb1 <- paste(comb1, sep="__", collapse="__")
        tmp_out$comb2 <- paste(comb2, sep="__", collapse="__")
        
        tmp_out$n_comb1 <- length(which(tmp_metadata$target_class == "comb1"))
        tmp_out$n_comb2 <- length(which(tmp_metadata$target_class == "comb2"))

        if (dim(out_df)[1] == 0) {
            
            out_df <- tmp_out
            
        } else {
            
            out_df <- rbind(out_df, tmp_out)
            
        }
        
    }
    
}

write_tsv(out_df, paste0(out_dir, "out_adonis.tsv"))

In [11]:
tmp_out

,Df,SumOfSqs,R2,F,Pr(>F),rownames,comb,comb1,comb2,n_comb1,n_comb2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
target_class,1,4.176661,0.007335388,15.67333,0.46,target_class,stage_class,control,adenoma,1471,652
Residual,2121,565.208413,0.992664612,NA,NA,Residual,stage_class,control,adenoma,1471,652
Total,2122,569.385074,1.000000000,NA,NA,Total,stage_class,control,adenoma,1471,652
